In [3]:
import pandas as pd
import psycopg2

conn = psycopg2.connect("dbname=postgres user=postgres password=postgres host=localhost")
cur = conn.cursor()

# Poblar tablas de dimensiones
dim_queries = [
    "INSERT INTO dim_clientes (id_cliente, nombre) SELECT DISTINCT id, UPPER(nombre) FROM clientes;",
    "INSERT INTO dim_productos (id_producto, nombre, categoria) SELECT DISTINCT id, UPPER(nombre), 'General' FROM productos;",
    "INSERT INTO dim_tiempo (fecha, dia, mes, anio) SELECT DISTINCT fecha, EXTRACT(DAY FROM fecha), EXTRACT(MONTH FROM fecha), EXTRACT(YEAR FROM fecha) FROM ordenes;",
    "INSERT INTO dim_corresponsales (id_corresponsal, nombre) SELECT DISTINCT id, UPPER(nombre) FROM corresponsales;"
]


for query in dim_queries:
    try:
        cur.execute(query)
        conn.commit()
    except Exception as e:
        print("Error:", e)

query = """
    INSERT INTO hechos_ventas (id_cliente, id_producto, id_corresponsal, fecha, total_pagado, cantidad)
    SELECT 
        o.id_cliente, 
        det.id_producto, 
        COALESCE(c.id, 1), 
        o.fecha, 
        ROUND((det.precio * det.cantidad), 2) AS total_pagado,
        det.cantidad
    FROM ordenes o
    JOIN detalles_ordenes det ON o.id = det.id_orden
    LEFT JOIN corresponsales c ON o.id_corresponsal = c.id;
    """

cur.execute(query)
conn.commit()

print("Data replicada correctamente en el Data Warehouse.")

# Cerrar conexión
cur.close()
conn.close()

Data replicada correctamente en el Data Warehouse.
